In [ ]:
import erroranalysis as ea
import numpy as np
from scipy.constants import k as k_b
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import os,sys,glob
import numpy as np
import pimcscripts.pimchelp as pimchelp
import pimcscripts.MCstat as MCstat
from collections import defaultdict
import argparse 
import subprocess
import pickle

#from joblib import Parallel, delayed
import multiprocessing
directory_paths = {
    r'$N_g = 0$': '../data/QMC/qmc_OUTPUT/NONovember12/',
    r'$N_g = 1$': '../data/QMC/qmc_OUTPUT/FirstNovember12/',
    r'$N_g = 2$': '../data/QMC/qmc_OUTPUT/SECONDNovember12/',
    r'$N_g = 3$':'../data/QMC/qmc_OUTPUT/THIRDNovember12/',
    r'$N_g = \infty$':'../data/QMC/qmc_OUTPUT/FULLNovember12/'
}
for key, value in directory_paths.items():
    print(f"Key: {key}, Value: {value}")

In [ ]:
#%run ~/.local/bin/merge.py -u -100 /lustre/isaac/scratch/gerwin2/local/pimc/build/OUTPUT/FULLSeptember3/
#directory_paths[r'$N_g = 0$']=value+'MERGED/'
print(directory_paths[r'$N_g = 0$'])

In [ ]:
def compressibility(N,N2,volume=1.0,k_B=1.0,T=1.0):
    pf = 1/(volume*k_B*T)
    return pf*(((N2 - (N**2)))/1)
def heatcapacity(E,E2,k_B=1.0,T=1.0):
    pf = 1/(k_B*(T**2))
    return pf*(((E2 - (E**2))))

In [ ]:
MERGED_files = True
if MERGED_files == False:
    folder_name = "MERGED"
    for key, value in directory_paths.items():
        directory = value
        if key == r'$N_g = 0$' or key == r'$N_g = \infty$':
            for i in range(-125, -14):
                %run ~/.local/bin/merge.py -u {i} {directory}
            #directory_paths[key]=value+'MERGED/'
    for key, value in directory_paths.items():
        directory = value
        if key == r'$N_g = 0$' or key == r'$N_g = \infty$':
            directory_paths[key]=value+'MERGED/'

In [ ]:
for key, value in directory_paths.items():
    print(f"Key: {key}, Value: {value}")

In [ ]:
data={}
Analysis = False
pickle_file_path = '../data/data.pkl'

if Analysis == True:
    all_file_names = os.listdir('../data/QMC/NONovember12/MERGED/')
    gce_estimator_files = [f for f in all_file_names if f.startswith('gce-estimator-')]
    num = len(gce_estimator_files)-3
    box_volume = 14.757072880486835*17.04*10#\AA^3
    volume = box_volume
    temperature = 1 #K
    skip = 500
    for key, value in directory_paths.items():
        all_file_names = os.listdir(value)
        gce_estimator_files = [f for f in all_file_names if f.startswith('gce-estimator-')]
        gce_estimator_files = pimchelp.sortFileNames(gce_estimator_files)
        data[key] ={'u':np.zeros([num],float),'aveN':np.zeros([num],float),'errN':np.zeros([num],float),'aveKappa':np.zeros([num],float),'errKappa':np.zeros([num],float),'aveC':np.zeros([num],float),'errC':np.zeros([num],float)}
        for i in range(num):
            j=i+3
            headers = pimchelp.getHeadersFromFile(value+'/'+gce_estimator_files[j])
            param = pimchelp.getFileNameParameters(gce_estimator_files[j])
            chem = float(param[4])
            estData = np.loadtxt(value+'/'+gce_estimator_files[j],ndmin=2)
            N = estData[:,headers.index('N')]
            E = estData[:,headers.index('E')]
            boot = ea.bootstrap(N)
            data[key]['u'][i] = chem
            data[key]['aveN'][i] = boot[0]
            data[key]['errN'][i] = boot[1]
            κ_avg, κ_err = ea.bootstrap_on_function(compressibility,N[skip:],N[skip:]**2,volume=volume,k_B=1,T=temperature)
            data[key]['aveKappa'][i] = κ_avg
            data[key]['errKappa'][i] = κ_err
            C_avg, C_err = ea.bootstrap_on_function(heatcapacity,E,E**2,k_B=1,T=temperature)
            data[key]['aveC'][i] = C_avg
            data[key]['errC'][i] = C_err
            with open(pickle_file_path, 'wb') as file:
                pickle.dump(data, file)
with open(pickle_file_path, 'rb') as file:
    data = pickle.load(file)

In [ ]:
print(data['$N_g = 0$']['u'][:])
print(data['$N_g = 0$']['aveN'][70:])

In [ ]:
#plt.rcParams.update(plt.rcParamsDefault)
plt.errorbar(data[r'$N_g = \infty$']['u'],data[ r'$N_g = \infty$']['aveN'],yerr=data[ r'$N_g = \infty$']['errN'],color='blue',label=r'$N_g = \infty$')
#plt.errorbar(data[r'$N_g = 1$']['u'],data[ r'$N_g = 1$']['aveN'],yerr=data[ r'$N_g = 1$']['errN'],color='black',label=r'$N_g = 3$')
plt.errorbar(data[ r'$N_g = 0$']['u'],data[ r'$N_g = 0$']['aveN'],yerr=data[ r'$N_g = 0$']['errN'],color='red',label=r'$N_g = 0$')

plt.ylabel(r'$N$',fontsize=10)
plt.xlabel(r'$\mu$(K)',fontsize=10)
#plt.xlim(-40,-15)
#plt.ylim(25,50)
plt.legend(loc='upper left')
plt.show()

In [ ]:
blue_color = (0, 0, 1)      # (R, G, B) = (0, 0, 1) for blue
black_color = (0, 0, 0)     # (R, G, B) = (0, 0, 0) for black

# Plot with error bars and custom RGB colors
plt.errorbar(data[r'$N_g = 0$']['u'], data[r'$N_g = 0$']['aveKappa'], yerr=data[r'$N_g = 0$']['errKappa'], color=blue_color, label=r'$N_g = 0$')
plt.errorbar(data[r'$N_g = \infty$']['u'], data[r'$N_g = \infty$']['aveKappa'], yerr=data[r'$N_g = \infty$']['errKappa'], color=black_color, label=r'$N_g = \infty$')
#plt.errorbar(data[ r'$N_g = 3$']['u'],data[ r'$N_g = 3$']['aveKappa'],yerr=data[ r'$N_g = 3$']['errKappa'],color='green')
#plt.errorbar(data[ r'$N_g = 2$']['u'],data[ r'$N_g = 2$']['aveKappa'],yerr=data[ r'$N_g = 2$']['errKappa'],color='black')
#plt.errorbar(data[ r'$N_g = 1$']['u'],data[ r'$N_g = 1$']['aveKappa'],yerr=data[ r'$N_g = 1$']['errKappa'],color='orange')
#plt.ylim(0,2e-6)
#plt.ylim(0,2.5e-6)
plt.xlim(-120,-30)
#plt.yscale('log')
plt.ylim(0,.001)
plt.ylabel(r'Compressibility $\kappa$ $(K^{-1}Å^{-3})$',fontsize=12)
plt.xlabel(r'$\mu$ (K)',fontsize=12)
plt.legend(loc='upper left')
#plt.savefig('../figures/Fig5.pdf', format='pdf')
plt.show()

In [ ]:
#plt.rcParams.update(plt.rcParamsDefault)
plt.errorbar(data[r'$N_g = \infty$']['u'],data[ r'$N_g = \infty$']['aveC'],yerr=data[ r'$N_g = \infty$']['aveC'],color='blue',label=r'$N_g = \infty$')
#plt.errorbar(data[ r'$N_g = 3$']['u'],data[ r'$N_g = 3$']['aveC'],yerr=data[ r'$N_g = 3$']['aveC'],color='green')
#plt.errorbar(data[ r'$N_g = 2$']['u'],data[ r'$N_g = 2$']['aveC'],yerr=data[ r'$N_g = 2$']['aveC'],color='black')
#plt.errorbar(data[ r'$N_g = 1$']['u'],data[ r'$N_g = 1$']['aveC'],yerr=data[ r'$N_g = 1$']['aveC'],color='orange')

plt.errorbar(data[ r'$N_g = 0$']['u'],data[ r'$N_g = 0$']['aveC'],yerr=data[ r'$N_g = 0$']['aveC'],color='red',label=r'$N_g = 0$')
plt.ylim(0,5e4)
plt.xlim(-120,-15)
plt.ylabel(r'K/J (Joules/K)',fontsize=12)
plt.xlabel(r'$\mu$(K)',fontsize=12)
plt.legend(loc='upper right')
plt.savefig('../figures/Fig8.eps', format='eps')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Your existing data and plotting code...

# Define RGB values for blue and black
blue_color = (0, 0, 1)      # (R, G, B) = (0, 0, 1) for blue
black_color = (0, 0, 0)     # (R, G, B) = (0, 0, 0) for black

# Create the main plot
plt.figure(figsize=(10, 6))

# Plot with error bars and custom RGB colors
plt.errorbar(data[r'$N_g = 0$']['u'], data[r'$N_g = 0$']['aveKappa'], yerr=data[r'$N_g = 0$']['errKappa'], color=blue_color, label=r'$N_g = 0$')
plt.errorbar(data[r'$N_g = \infty$']['u'], data[r'$N_g = \infty$']['aveKappa'], yerr=data[r'$N_g = \infty$']['errKappa'], color=black_color, label=r'$N_g = \infty$')
plt.xlim(-120, -15)
#plt.ylim(0, 0.001)
plt.ylabel(r'Compressibility $\kappa$ $(K^{-1}Å^{-3})$', fontsize=12)
plt.xlabel(r'$\mu$ (K)', fontsize=12)
plt.legend(loc='upper left')

# Create the inset axes in the top right corner
axins = plt.axes([0.35, 0.45, 0.3, 0.3])  # (left, bottom, width, height)

# Plot the data in the inset
axins.errorbar(data[r'$N_g = 0$']['u'], data[r'$N_g = 0$']['aveKappa'], yerr=data[r'$N_g = 0$']['errKappa'], color=blue_color, label=r'$N_g = 0$')
axins.errorbar(data[r'$N_g = \infty$']['u'], data[r'$N_g = \infty$']['aveKappa'], yerr=data[r'$N_g = \infty$']['errKappa'], color=black_color, label=r'$N_g = \infty$')
axins.set_xlim(-120, -30)
axins.set_ylim(0, 0.001)

# Customize the inset appearance
axins.set_xlabel(r'$\mu$ (K)')
axins.set_ylabel(r'Compressibility $\kappa$ $(K^{-1}Å^{-3})$')
plt.savefig('../figures/Fig5.svg', format='svg')
plt.savefig('../figures/Fig5.png', format='png')
